
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.1/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

def get_detection_data(set_name):

  data_dirs = '{}/data'.format(BASE_DIR)
  dataset = []
  curren_img = "START"
  first_img = True

  
  if set_name == "train":
    json_file = os.path.join(data_dirs, "train.json")
    with open(json_file) as f:
          imgs_anns = json.load(f)
    objs = []
    jsonLen = len(imgs_anns)
    for index in range(jsonLen):
      value = imgs_anns[index]
      if curren_img != value["file_name"]:
        if first_img == False:
          dataset.append(record)
        objs = []
        curren_img = value["file_name"]
        record = {}
        filename = os.path.join('{}/data/train'.format(BASE_DIR), value["file_name"])
        height, width = cv2.imread(filename).shape[:2]
        record["file_name"] = filename
        record["image_id"] = value["image_id"]
        record["height"] = height
        record["width"] = width
        obj = {
              "bbox": value["bbox"],
              "bbox_mode": BoxMode.XYWH_ABS,
              "segmentation": value["segmentation"],
              "category_id": 0,
            }
        objs.append(obj)
        record["annotations"] = objs
        first_img = False

      else:
        obj = {
              "bbox": value["bbox"],
              "bbox_mode": BoxMode.XYWH_ABS,
              "segmentation": value["segmentation"],
              "category_id": 0,
            }
        objs.append(obj)
        record["annotations"] = objs
        if (index == jsonLen-1):
          dataset.append(record)
  elif set_name == "test":
  #else:
    for filename in os.listdir('{}/data/test'.format(BASE_DIR)):
      record = {}
      filePath = os.path.join('{}/data/test'.format(BASE_DIR), filename)
      height, width = cv2.imread(filePath).shape[:2]
      record["file_name"] = filePath
      record["height"] = height
      record["width"] = width
      record["annotations"] = []
      dataset.append(record)
  else: #set_name == "val"
    val_count = 0
    json_file = os.path.join(data_dirs, "train.json")
    with open(json_file) as f:
          imgs_anns = json.load(f)
    objs = []
    jsonLen = len(imgs_anns)
    for index in range(jsonLen):
      value = imgs_anns[index]
      if curren_img != value["file_name"]:
        if first_img == False:
          dataset.append(record)
          val_count = val_count + 1
          if val_count == 20:
            return dataset
        objs = []
        curren_img = value["file_name"]
        record = {}
        filename = os.path.join('{}/data/train'.format(BASE_DIR), value["file_name"])
        height, width = cv2.imread(filename).shape[:2]
        record["file_name"] = filename
        record["image_id"] = value["image_id"]
        record["height"] = height
        record["width"] = width
        obj = {
              "bbox": value["bbox"],
              "bbox_mode": BoxMode.XYWH_ABS,
              "segmentation": value["segmentation"],
              "category_id": 0,
            }
        objs.append(obj)
        record["annotations"] = objs
        first_img = False

      else:
        obj = {
              "bbox": value["bbox"],
              "bbox_mode": BoxMode.XYWH_ABS,
              "segmentation": value["segmentation"],
              "category_id": 0,
            }
        objs.append(obj)
        record["annotations"] = objs
        if (index == jsonLen-1):
          dataset.append(record)

  return dataset

  

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train","test","val"]:
    DatasetCatalog.register("plane_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("plane_" + d).set(thing_classes=["planes"])
plane_metadata = MetadataCatalog.get("plane_train")


In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset_dicts = get_detection_data("train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
clean_DatasetCatalog = 1
if clean_DatasetCatalog == 2:
  DatasetCatalog.remove("plane_train")
  DatasetCatalog.remove("plane_test")
  DatasetCatalog.remove("plane_val")

### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

##############################Reference of below code from:#########################################################
##########https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=7unkuuiqLdqd ############# 
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # batch size
cfg.SOLVER.BASE_LR = 0.0003  # LR
cfg.SOLVER.MAX_ITER = 3000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only have plane

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)



### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/CMPT_CV_lab3/output

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
##############################Reference of below code from:#########################################################
##########https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=7unkuuiqLdqd ############# 
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.3, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#evaluator = COCOEvaluator("plane_val", ("bbox","segm"),output_dir="{}/output/".format(BASE_DIR))
evaluator = COCOEvaluator("plane_train", cfg,False,output_dir="{}/output/".format(BASE_DIR))
val_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''
#torch.cuda.empty_cache()

## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
#pre-load the data to aviod the time waste caused by meaningless repeat reading img file
trainRawData = get_detection_data("train")
trainFixedData = []
for d in trainRawData:
  record = {}
  record['file_name'] = d['file_name']
  img = cv2.imread(record['file_name'])
  objs=[]
  for idx in range(len(d['annotations'])):
    img_ann = d['annotations'][idx]
    img_bbox = img_ann['bbox']
    img_seg = img_ann['segmentation']
    box_x1,box_y1,box_width,box_high =[int(num) for num in img_bbox]
    # box_x1,box_y1: Top Left
    # box_x2,box_y2: Bottom right
    box_x2 = box_x1 + box_width
    box_y2 = box_y1 + box_high

    #process segmentation
    img_seg_new = []
    for i in range(0,len(img_seg[0])-1,2):
      img_seg_new.append(img_seg[0][i] - box_x1)
      img_seg_new.append(img_seg[0][i+1] - box_y1)
      #img_seg_new.append(img_seg[0][i])
      #img_seg_new.append(img_seg[0][i+1])
    img_seg_new = [img_seg_new]

    #crop img [y0:y1, x0:x1]
    obj_img = img[box_y1:box_y2, box_x1:box_x2]
    obj_img = cv2.resize(obj_img,(128,128),interpolation = cv2.INTER_AREA)

    #make mask
    
    obj_mask = detectron2.utils.visualizer.GenericMask(img_seg_new,box_high,box_width).mask
    #obj_mask = detectron2.utils.visualizer.GenericMask(img_ann['segmentation'][0],box_width,box_high).mask
    #print(obj_mask)
    #print(obj_mask.shape)
    obj_mask = cv2.resize(obj_mask,(128,128),interpolation = cv2.INTER_AREA)

    obj={
        "img":obj_img,
        "mask":obj_mask,
    }
    objs.append(obj)
  record["annotations"]=objs
  trainFixedData.append(record)

  




In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
#trainData = get_detection_data('train')
def get_instance_sample(data, idx, img=None):
  for index1 in range(len(trainFixedData)):
    if trainFixedData[index1]["file_name"] == data['file_name']:
      obj_img = trainFixedData[index1]["annotations"][idx]["img"]
      obj_mask = trainFixedData[index1]["annotations"][idx]["mask"]
      return obj_img, obj_mask
  return obj_img, obj_mask

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
    #get: set_name=train，data_list=plane_train
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    
    #instance_map=[[1,1],[1,2],[1,3],....[3,1],[3,2],[3,3]]
    index = idx[1]

    obj_img, obj_mask = get_instance_sample(data,index)
    img,mask = self.numpy_to_tensor(obj_img,obj_mask)
    mask = mask.reshape((1,128,128))
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("plane_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            conv(out_ch, out_ch),   #Add by FangKK
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)
        self.BN = nn.BatchNorm2d(out_ch) #Add by FangKK

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        y = self.BN(y) #Add by FangKK
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        self.input_conv = conv(3, 4)
        self.down1 = down(4, 8)
        self.down2 = down(8, 16)
        self.down3 = down(16, 32)
        self.down4 = down(32, 64)
        self.down5 = down(64, 128)
        self.down6 = down(128, 256)
        
        # Decoder
        self.up1 = up(256, 128)
        self.up2 = up(128, 64)
        self.up3 = up(64, 32)
        self.up4 = up(32, 16)
        self.up5 = up(16, 8)
        self.up6 = up(8, 4)
        #self.up7 = up(8, 4)
        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function
        
    def forward(self, input):
      y = self.input_conv(input)
      y = self.down1(y)
      y = self.down2(y)
      y = self.down3(y)
      y = self.down4(y)
      y = self.down5(y)
      y = self.down6(y)
      y = self.up1(y)
      y = self.up2(y)
      y = self.up3(y)
      y = self.up4(y)
      y = self.up5(y)
      y = self.up6(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 150
batch_size = 4
#learning_rate = 0.01
#learning_rate = 0.007 ,Mean IoU: 0.8436842446572008 Epoch: 29
#learning_rate = 0.0005 ,Mean IoU: 0.7198967454230489, Epoch: 49, Loss: 0.19871610403060913
#learning_rate = 0.0005 ,Mean IoU: 0.7669054685873025, Epoch: 79, Loss: 0.17480936646461487
#learning_rate = 0.001 ,Mean IoU: 0.8190553504369219, Epoch: 79, Loss: 0.141431525349617
#learning_rate = 0.002 ,Mean IoU: 0.8564498716450435, Epoch: 99, Loss: 0.10545563697814941
#learning_rate = 0.003 ,Mean IoU: 0.8740110476560341, Epoch: 99, Loss: 0.09390881657600403
#learning_rate = 0.003 ,Mean IoU: 0.8916377923312807, Epoch: 149, Loss: 0.08267544955015182
learning_rate = 0.003
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

count = 0
total_iou = 0

origImg_list=[]
predic_mask_list=[]
grdTrue_mask_list=[]

for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1)
    pred = model(img)
    new_batch_size = pred.shape[0]
    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    #pred.shape: torch.Size([batch_size, 1, 128, 128])
    #mask.shape: torch.Size([batch_size, 1, 1,  128, 128])
    #img.shape: torch.Size([batch_size, 3, 128, 128])
    
    for i in range(new_batch_size):
      count = count +1

      origImg = img[i].cpu()
      origImg = transforms.ToPILImage()(origImg)
      origImg = np.array(origImg)
      origImg_list.append(origImg)

      predic_mask = pred[i][0].cpu()
      predic_mask = np.array(predic_mask)
      predic_mask = sigmoid(predic_mask)
      predic_mask = np.rint(predic_mask)
      predic_mask_list.append(predic_mask)


      grdTreu_mask = mask[i][0][0].cpu()
      grdTreu_mask = grdTreu_mask.numpy()
      grdTrue_mask_list.append(grdTreu_mask)


      mask_intersection = predic_mask*grdTreu_mask
      mask_union = predic_mask+grdTreu_mask
      mask_intersection_count = np.count_nonzero(mask_intersection)
      mask_union_count =np.count_nonzero(mask_union)
      if mask_union_count != 0:
        total_iou = total_iou + (mask_intersection_count/mask_union_count)
      else:
        total_iou = total_iou + 0

print("\n #images: {}, Mean IoU: {}".format(count, total_iou/count))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
import matplotlib.pyplot as plt
num = 3
for i in range(num):
  idxEnd = len(origImg_list)-1
  idx = random.randint(0,idxEnd)
  cv2_imshow(origImg_list[idx])
  cv2_imshow(predic_mask_list[idx]*255)
  cv2_imshow(grdTrue_mask_list[idx]*255)


## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers

not_use_seg = False

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def get_prediction_mask(data,setType):
  #data is "plane_train"[i]
  #data['image_id']= P0000
  img_height = data['height']
  img_width = data['width']
  segmentation_total = []
  pred_mask= np.zeros((img_height, img_width))
  gt_mask= np.zeros((img_height, img_width))
  img = cv2.imread(data["file_name"])

  if (setType == 'train') and (not_use_seg == False):
    intensity = int(200/len(data["annotations"]))
    #gt_mask:
    #get the segmentation of all plane
    for idx in range(len(data["annotations"])):
      segmentation = []
      segmentation.append(data["annotations"][idx]["segmentation"][0])
      #print(segmentation)
      temp_gt_mask = detectron2.utils.visualizer.GenericMask(segmentation,img_height,img_width).mask
      temp_gt_mask = temp_gt_mask * intensity*(idx+1)
      temp_gt_mask[temp_gt_mask!=0] += 50
      gt_mask = np.maximum(temp_gt_mask,gt_mask)
    #get the gt_mask
    #gt_mask = detectron2.utils.visualizer.GenericMask(segmentation_total,img_height,imh_width).mask
    #print(gt_mask.shape)
    gt_mask = torch.tensor(gt_mask, device=torch.device('cuda'))
    # for tfData in trainFixedData:
    #   if (tfData["file_name"] == data["file_name"]):
    #   break

    #pred_mask:
    for idx in range(len(data["annotations"])):
      box_x1,box_y1,box_width,box_high =[int(num) for num in data["annotations"][idx]["bbox"]]
      # box_x1,box_y1: Top Left
      # box_x2,box_y2: Bottom right
      box_x2 = box_x1 + box_width
      box_y2 = box_y1 + box_high

      obj_img = img[box_y1:box_y2, box_x1:box_x2]
      obj_img = cv2.resize(obj_img,(128,128),interpolation = cv2.INTER_AREA)
      obj_img = torch.tensor(transforms.ToTensor()(obj_img), device=torch.device('cuda'))
      pred_obj_img_mask = model(torch.unsqueeze(obj_img,0))
      #print(pred_obj_img_mask)
      #print(pred_obj_img_mask.shape)
      pred_obj_img_mask = pred_obj_img_mask.cpu()
      #print(pred_obj_img_mask)
      pred_obj_img_mask = pred_obj_img_mask.detach().numpy()[0][0]
      #print(pred_obj_img_mask.shape)
      #print(pred_obj_img_mask)
      pred_obj_img_mask = cv2.resize(pred_obj_img_mask,(box_width,box_high),interpolation = cv2.INTER_AREA)
      #print(pred_obj_img_mask)
      pred_obj_img_mask = sigmoid(pred_obj_img_mask)
      pred_obj_img_mask = np.rint(pred_obj_img_mask)
      pred_obj_img_mask = pred_obj_img_mask * intensity*(idx+1)
      pred_obj_img_mask[pred_obj_img_mask!=0] += 50
      #print(pred_obj_img_mask)
      tempMap = pred_mask[box_y1:box_y2,box_x1:box_x2]
      tempMap = np.maximum(tempMap,pred_obj_img_mask)
      pred_mask[box_y1:box_y2,box_x1:box_x2] = tempMap
    pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'))
  else:
    
    #Use part1 get bbox:
    pred_outputs = predictor(img)
    #print(pred_outputs)
    bboxs = pred_outputs["instances"]
    bbox_list = []
    #print(len(bbox_list))
    if len(bboxs) != 0:
      intensity = int(200/len(bboxs))
    else:
      pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'))
      gt_mask = torch.tensor(gt_mask, device=torch.device('cuda'))
      return img, gt_mask, pred_mask

    for i in range(len(bboxs)):
      bbox = bboxs[i].pred_boxes
      bbox = bbox.tensor[0].cpu()
      x1,y1,x2,y2 = bbox
      box_x1 = int(x1)
      box_y1 = int(y1)
      box_x2 = int(x2)
      box_y2 = int(y2)
      box_width = box_x2 - box_x1
      box_high = box_y2 - box_y1
    # intensity = int(200/len(data["annotations"]))
      obj_img = img[box_y1:box_y2, box_x1:box_x2]
      obj_img = cv2.resize(obj_img,(128,128),interpolation = cv2.INTER_AREA)
      obj_img = torch.tensor(transforms.ToTensor()(obj_img), device=torch.device('cuda'))
      pred_obj_img_mask = model(torch.unsqueeze(obj_img,0))
      pred_obj_img_mask = pred_obj_img_mask.cpu()
      pred_obj_img_mask = pred_obj_img_mask.detach().numpy()[0][0]
      pred_obj_img_mask = cv2.resize(pred_obj_img_mask,(box_width,box_high),interpolation = cv2.INTER_AREA)
      pred_obj_img_mask = sigmoid(pred_obj_img_mask)
      pred_obj_img_mask = np.rint(pred_obj_img_mask)
      pred_obj_img_mask = pred_obj_img_mask * intensity*(i+1)
      pred_obj_img_mask[pred_obj_img_mask!=0] += 50
      tempMap = pred_mask[box_y1:box_y2,box_x1:box_x2]
      tempMap = np.maximum(tempMap,pred_obj_img_mask)
      pred_mask[box_y1:box_y2,box_x1:box_x2] = tempMap
    pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'))
    gt_mask = torch.tensor(gt_mask, device=torch.device('cuda'))


  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
my_data_list_test = DatasetCatalog.get("plane_{}".format('test'))


In [ ]:
for i in np.random.randint(0,len(my_data_list_test),3):
    sample = my_data_list_test[i]
    sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
    img, true_mask, pred_mask = get_prediction_mask(sample,'test')
    cv2_imshow((img))
    #cv2_imshow((true_mask.cpu().numpy()))
    cv2_imshow((pred_mask.cpu().numpy()))

In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("plane_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample,'train')
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("plane_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample,'test')
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
#Run the Data Loader from part1 !!!

### Network

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

##############################Reference of below code from:#########################################################
##########https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=7unkuuiqLdqd ############# 
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # batch size
cfg.SOLVER.BASE_LR = 0.0003  # LR
cfg.SOLVER.MAX_ITER = 3000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only have plane


### Training

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
##############################Reference of below code from:#########################################################
##########https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=7unkuuiqLdqd ############# 
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''

from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.3, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
#evaluator = COCOEvaluator("plane_val", ("bbox","segm"),output_dir="{}/output/".format(BASE_DIR))
evaluator = COCOEvaluator("plane_train", cfg,False,output_dir="{}/output/".format(BASE_DIR))
val_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))